In [6]:
# import win32com.client

# def list_outlook_stores():
#     outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
#     stores = outlook.Stores

#     print("Stores found in Outlook.Stores:")
#     for store in stores:
#         print(f"  DisplayName: {store.DisplayName}")
#     print()

# if __name__ == "__main__":
#     list_outlook_stores()


Stores found in Outlook.Stores:
  DisplayName: nihat.yumer@outlook.com
  DisplayName: n.yumer@my.bpp.com
  DisplayName: afizenihat@outlook.com
  DisplayName: nyumer@dlrmn.onmicrosoft.com



In [ ]:
import DataExtraction
import DataAnalysis

def main():
    # 1. Specify which mailbox/store to read, exactly matching 'store.DisplayName'
    # mailbox_name = "Nihat.Yumer@nhft.nhs.uk"  
    mailbox_name = "n.yumer@my.bpp.com"  
    subfolder_name = "ROMIC"       # must match exactly a subfolder under the Inbox

    # 2. Read emails from the subfolder
    DataExtraction.read_outlook_subfolder_stores(mailbox_name, subfolder_name)
    
    all_bodies = DataExtraction.read_outlook_subfolder_stores(mailbox_name, subfolder_name)
    print("All bodies (no signature):")
    for i, body_text in enumerate(all_bodies, start=1):
        print(f"Body #{i}:\n{body_text}")
        print("-" * 40)

if __name__ == "__main__":
    main()



--- Reading up to 5 emails from subfolder 'ROMIC' in mailbox 'n.yumer@my.bpp.com' ---
Email #1
  From:    Ansell Stuart
  Received: 2024-12-20 17:17:20.409000+00:00
  Body (no signature):

 X6 24” screens – MASH @ CJC
----------------------------------------
Email #2
  From:    Melay Connor
  Received: 2024-12-20 15:37:20.598000+00:00
  Body (no signature):

 Inventory of Hardware Collected from warehouse  1 x 5540  

  Asset details of devices collected (if required)     Asset 076975
----------------------------------------
Email #3
  From:    Ansell Stuart
  Received: 2024-12-20 14:30:01.684000+00:00
  Body (no signature):

 X1 3000 - nhft-073807
----------------------------------------
Email #4
  From:    Baramasimbe Ronnie
  Received: 2024-12-20 12:12:07.829000+00:00
  Body (no signature):

 Hi,

 

I have booked out the following items:

 

4 x monitors for SCTASK0035461, SCTASK0031985 and CJC 

 

With
----------------------------------------
Email #5
  From:    Melay Connor
  Re

TypeError: 'NoneType' object is not iterable